In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as f
from PIL import Image
import os
from sklearn import metrics
import glob

In [5]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [6]:
label_encoder=LabelEncoder()

In [7]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [8]:
dataframe=df

In [9]:

class_counts = df['Class Name'].value_counts()

print("0 sayısı:", class_counts.get(0, 0))  
print("1 sayısı:", class_counts.get(1, 0))   


0 sayısı: 15978
1 sayısı: 35672


In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51650 entries, 0 to 51649
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  51650 non-null  int64  
 1   ax          51650 non-null  float64
 2   ay          51650 non-null  float64
 3   az          51650 non-null  float64
 4   gx          51650 non-null  float64
 5   gy          51650 non-null  float64
 6   gz          51650 non-null  float64
 7   timestamp   51650 non-null  object 
 8   Class Name  51650 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 3.3+ MB


In [11]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

# son zaman adım etiketi alındı , bu değiştirilebilir , istatistik olarak daha fazla olanı tercih ediyorum .

In [15]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class CNNDataset(Dataset):
    
    def __init__ (self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length
        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])
    
    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1
    
    def __getitem__(self, idx):
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        #son zaman adımı etiketi alındı . bu değiştirilebilir
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32).permute(1, 0)   
        label_tensor = torch.tensor(label, dtype=torch.long)
        
        return features_tensor, label_tensor



In [16]:
dataset = CNNDataset(dataframe)

 
labels = dataset.dataframe['Class Name'].values[:len(dataset)]   

 
indices = list(range(len(dataset)))

 
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)
 
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [17]:
for x,y  in train_loader:
    print(x.shape)
    print(y.shape)


torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 6, 25])
torch.Size([32])
torch.Size([32, 

KeyboardInterrupt: 

class ImprovedCNNModel1D(nn.Module):
    def __init__(self, input_length):
        super().__init__()
        self.conv1 = nn.Conv1d(6, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        
        # Calculate the size of the feature map after convolution and pooling layers
        def conv1d_output_size(length, kernel_size=3, padding=1, stride=1):
            return (length + 2*padding - (kernel_size - 1) - 1) // stride + 1

        def pool1d_output_size(length, kernel_size=2, stride=2):
            return (length - (kernel_size - 1) - 1) // stride + 1
        
        l = input_length
        l = conv1d_output_size(l)  # conv1
        l = pool1d_output_size(l)  # pool1
        l = conv1d_output_size(l)  # conv2
        l = pool1d_output_size(l)  # pool2
        l = conv1d_output_size(l)  # conv3
        l = pool1d_output_size(l)  # pool3
        
        self.flattened_size = 128 * l  # 128 channels after conv3 and final pooled length

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)  # Output a single value for binary classification

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, self.flattened_size)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid to output a probability between 0 and 1
        return x


In [ ]:
class ImprovedCNNModel1D(nn.Module):
    def __init__(self, input_length, dropout_rate=0.5):  
        super().__init__()
        
        # Conv1d katmanları
        self.conv1 = nn.Conv1d(6, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        
        # MaxPool katmanı
        self.pool = nn.MaxPool1d(2)
        
        # Dropout 
        self.dropout = nn.Dropout(dropout_rate)  # Dropout katmanı  
        
        # Calculate the size of the feature map after convolution and pooling layers
        def conv1d_output_size(length, kernel_size=3, padding=1, stride=1):
            return (length + 2*padding - (kernel_size - 1) - 1) // stride + 1

        def pool1d_output_size(length, kernel_size=2, stride=2):
            return (length - (kernel_size - 1) - 1) // stride + 1
        
        l = input_length
        l = conv1d_output_size(l)  # conv1
        l = pool1d_output_size(l)  # pool1
        l = conv1d_output_size(l)  # conv2
        l = pool1d_output_size(l)  # pool2
        l = conv1d_output_size(l)  # conv3
        l = pool1d_output_size(l)  # pool3
        
        self.flattened_size = 128 * l   

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)   

    def forward(self, x):
         
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.dropout(x)   
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.dropout(x)   
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.dropout(x)   
        
         
        x = x.view(-1, self.flattened_size)
        
         
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)   
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)  
        x = torch.sigmoid(self.fc3(x))  # Sigmoid ile 0-1 arası olasılık
        return x


In [30]:
criterion = nn.BCELoss()  # Binary Cross-Entropy loss


In [31]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

model = ImprovedCNNModel1D(input_length=25)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [32]:
if torch.cuda.is_available():
    device=torch.device("cuda")

In [33]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

model = ImprovedCNNModel1D(input_length=25).to(device)   
criterion = nn.BCELoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        #forward pass 
        outputs = model(data)
        outputs = outputs.squeeze(1)  

        # Calculate loss
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = (outputs >= 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")



Epoch [1/10], Loss: 0.4188, Accuracy: 80.62%
Epoch [2/10], Loss: 0.2559, Accuracy: 89.17%
Epoch [3/10], Loss: 0.1655, Accuracy: 93.30%
Epoch [4/10], Loss: 0.1151, Accuracy: 95.50%
Epoch [5/10], Loss: 0.0885, Accuracy: 96.59%
Epoch [6/10], Loss: 0.0772, Accuracy: 97.20%
Epoch [7/10], Loss: 0.0645, Accuracy: 97.69%
Epoch [8/10], Loss: 0.0591, Accuracy: 97.85%
Epoch [9/10], Loss: 0.0556, Accuracy: 97.97%
Epoch [10/10], Loss: 0.0497, Accuracy: 98.27%


In [35]:

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad() 
        outputs = model(data)
        outputs = outputs.squeeze(1)  
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = (outputs >= 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch [1/10], Loss: 0.0468, Accuracy: 98.36%
Epoch [2/10], Loss: 0.0470, Accuracy: 98.33%
Epoch [3/10], Loss: 0.0430, Accuracy: 98.48%
Epoch [4/10], Loss: 0.0406, Accuracy: 98.56%
Epoch [5/10], Loss: 0.0384, Accuracy: 98.66%
Epoch [6/10], Loss: 0.0400, Accuracy: 98.65%
Epoch [7/10], Loss: 0.0352, Accuracy: 98.78%
Epoch [8/10], Loss: 0.0373, Accuracy: 98.79%
Epoch [9/10], Loss: 0.0357, Accuracy: 98.74%
Epoch [10/10], Loss: 0.0347, Accuracy: 98.81%


In [37]:
model.eval()
correct = 0
total = 0

with torch.no_grad():  
    for batch_idx, (data, labels) in enumerate(test_loader):
        data, labels = data.to(device), labels.to(device)

        outputs = model(data)
        outputs = outputs.squeeze(1)  # Bce loss için

        predicted = (outputs >= 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.67%
